In [1]:
import pandas as pd

# 2. Extraer información de las películas

En este notebook vamos a extraer toda la información que podamos de las películas. Para ello, hemos descargado un dataset con toda la información de IMDB de 5000 películas. Este dataset se encuentra en https://www.kaggle.com/carolzhangdc/imdb-5000-movie-dataset.

## 2.1. Cargamos los datasets

Cargamos los datasets de las películas que existen en *MovieLens* y las películas que tenemos en el dataset de los metadatos de *IMBD*.

In [2]:
imdb_df = pd.read_csv('data/original_dataset/movie_metadata.csv')
imdb_df.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


In [3]:
# Cargamos los links en IMDB de las películas
movielens_df = pd.read_csv('data/original_dataset/ml-latest-small/links.csv')
movielens_df.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [4]:
# Filtramos las películas que hay en ratings
ratings_df = pd.read_csv('data/experiment_data/ratings.csv')
movielens_df = movielens_df[movielens_df['movieId'].isin(set(ratings_df['movieId']))]
movielens_df.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


## 2.2. Obtener las películas que hay en ambos dataset

A contianuación, nos quedamos con las películas que se encuentran en ambos datasets. Para ello, vamos a obtener el identificador de la películas en *IMDB* y quedarnos con aquellas paelículas cuyo identificador de *IMDB* se encuentran en los dos datasets.

In [5]:
def get_id(row):
    '''
    Función para obtener ek identificador de IMDB
    a partir de su URL.
    '''
    return int(row[28:].split('/')[0])

In [6]:
# Creamos las columna 'imdbId' en el datasets de IMDB
imdb_df['movie_imdb_id'] = imdb_df.apply(lambda row: get_id(row['movie_imdb_link']), axis=1)

# Quitamos de los 2 datasets las películas que no están en alguno de los datasets
movielens_df = movielens_df[movielens_df['imdbId'].isin(imdb_df['movie_imdb_id'].values)]
imdb_df = imdb_df[imdb_df.movie_imdb_id.isin(movielens_df['imdbId'].values)]

# Eliminamos las filas duplicadas
imdb_df.drop_duplicates(subset=['movie_imdb_id'], inplace=True)

# Si el hay valor NaN lo convertimos a 0
imdb_df.fillna(0, inplace=True)
imdb_df.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes,movie_imdb_id
76,Color,Kevin Reynolds,91.0,176.0,58.0,60.0,Rick Aviles,711.0,88246220.0,Action|Adventure|Sci-Fi|Thriller,...,English,USA,PG-13,175000000.0,1995.0,60.0,6.1,1.85,0,114898
288,Color,James Cameron,210.0,153.0,0.0,539.0,Jenette Goldstein,780.0,204843350.0,Action|Sci-Fi,...,English,USA,R,102000000.0,1991.0,604.0,8.5,2.35,13000,103064
291,Color,James Cameron,94.0,141.0,0.0,618.0,Tia Carrere,2000.0,146282411.0,Action|Comedy|Thriller,...,English,USA,R,115000000.0,1994.0,1000.0,7.2,2.35,0,111503
309,Color,Joel Schumacher,144.0,121.0,541.0,680.0,Rene Auberjonois,920.0,184031112.0,Action|Adventure|Fantasy,...,English,USA,PG-13,100000000.0,1995.0,710.0,5.4,1.85,0,112462
350,Color,Renny Harlin,61.0,124.0,212.0,810.0,Frank Langella,1000.0,11000000.0,Action|Adventure|Comedy,...,English,USA,PG-13,98000000.0,1995.0,903.0,5.6,2.35,1000,112760


## 2.3. Limpiamos datos

Eliminamos los datos del datasets de *IMDB* que no necesitamos en el experimento. Además, la información de la duración y el año los pasamos a valores enteros. Por último, al dataset de *IMDB* le añadimos los identificadores de *MovieLens* de las películas que serán con las que trabajaremos.

In [7]:
def get_actors(row):
    return str(row['actor_1_name']) + '|' + str(row['actor_3_name']) + '|' + str(row['actor_3_name'])

In [8]:
# Juntamos los actores
imdb_df['stars'] = imdb_df.apply(lambda row: get_actors(row), axis=1)

# Eliminamos columnas
columns_to_drop = ['color', 
                   'num_critic_for_reviews', 
                   'director_facebook_likes', 
                   'actor_3_facebook_likes', 
                   'actor_2_facebook_likes', 
                   'actor_1_facebook_likes',
                   'num_user_for_reviews',
                   'movie_facebook_likes',
                   'gross',
                   'num_voted_users',
                   'cast_total_facebook_likes',
                   'facenumber_in_poster',
                   'movie_imdb_link',
                   'budget',
                   'imdb_score',
                   'aspect_ratio',
                   'plot_keywords',
                   'actor_1_name',
                   'actor_2_name',
                   'actor_3_name',
                   'content_rating']

imdb_df.drop(columns=columns_to_drop, inplace=True)

# Convertimos los datos de las columnas años y duración
imdb_df['title_year'] = imdb_df.apply(lambda row: str(int(row['title_year'])), axis=1)
imdb_df['duration'] = imdb_df.apply(lambda row: str(int(row['duration'])), axis=1)

# Añadimos los identificadores de las películas
imdb_df.sort_values(by=['movie_imdb_id'], ascending=True, inplace=True)
movielens_df.sort_values(by=['imdbId'], ascending=True, inplace=True)
imdb_df['id'] = movielens_df['movieId'].values
imdb_df.set_index('id', inplace=True)
imdb_df.reset_index(inplace=True)

imdb_df.drop(columns=['movie_imdb_id'], inplace=True)
imdb_df.head()

,id,director_name,duration,genres,movie_title,language,country,title_year,stars
0,594,William Cottrell,83,Animation|Family|Fantasy|Musical,Snow White and the Seven Dwarfs,English,USA,1937,Adriana Caselotti|Lucille La Verne|Lucille La ...
1,596,Norman Ferguson,88,Animation|Family|Fantasy|Musical,Pinocchio,English,USA,1940,Mel Blanc|Cliff Edwards|Cliff Edwards
2,599,Sam Peckinpah,144,Action|Adventure|Western,The Wild Bunch,English,USA,1969,William Holden|L.Q. Jones|L.Q. Jones
3,111,Martin Scorsese,110,Crime|Drama,Taxi Driver,English,USA,1976,Robert De Niro|Peter Boyle|Peter Boyle
4,260,George Lucas,125,Action|Adventure|Fantasy|Sci-Fi,Star Wars: Episode IV - A New Hope,English,USA,1977,Harrison Ford|Kenny Baker|Kenny Baker


## 2.4. Guardar datasets limpio

Guardamos el dataset en un fichero CSV para utilizar más adelante.

In [9]:
imdb_df.to_csv('data/experiment_data/movies.csv', index=False)

## 2.5. Filtrar peliculas en ratings CSV

Eliminamos las películas de ratings.csv que no están representadas en `movies_data_df`.

In [10]:
ratings_df = pd.read_csv('data/experiment_data/ratings.csv')
len(ratings_df)

17180

In [11]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [12]:
ratings_df = ratings_df[ratings_df['movieId'].isin(imdb_df.id.values)]
len(ratings_df)

11477

In [13]:
ratings_df.to_csv('data/experiment_data/ratings.csv', index=False)

## Siguiente Notebook

Ya tenemos los datasets del experimento preparados. A continuación, hay que descargar toda la información posible de las películas. Para ello, ejecute el notebook `vectores_binarios_peliculas`